In [2]:
from scipy import optimize
import scipy.stats as st
import numpy as np
import pandas as pd
import math
import sys
import matplotlib.pyplot as plt
import scipy.io
from sklearn.linear_model import LinearRegression
import os.path
from os import path
import netCDF4
from statsmodels.tsa.stattools import acf
import netCDF4 as nc
import warnings
warnings.filterwarnings("ignore")
from scipy import optimize
import scipy.stats as st
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import sys
import glob
import os
def piecewise_linear(x, x0, y0, k1, k2):
#   Compute optimization of fitting two connected linregs to data.
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])

def df_autocorr(df, lag=1, axis=0):
#   Compute full-sample column-wise autocorrelation for a DataFrame.
    return df.apply(lambda col: col.autocorr(lag), axis=axis)
"""
        optimize.curve_fit does the heavy lifting. p & e are 4-element series of results and error/uncertainties for:
        [0] : X or soil moisture value for breakpoint
        [1] : Y (T, flux, etc.) value for breakpoint
        [2] : Slope on left side of breakpoint
        [3] : slope on right side of breakpoint
        piecewise_linear is defined above - the function to optimize over
        Next 2 arguments are X and Y series (sorted on X) of daily data
        p0 is an optional first guess for each of the 4 predicted parameters
        bounds sets limits
"""        
pair_list=np.load('/homes/hhsu/02.InfoTheo/FluxNet/pair_list.npy')
pair_list

txtfile = '/homes/hhsu/02.InfoTheo/FluxNet/FluxNet_site_info.txt'
id = []     # The number of the site
name = []   # The name of the site
lat = []    # latitude of the site
lon = []    # lontitude of the site
lct = []    # IGBP classification of the site
gns = []    # Vegetation cover level of the site
pool = []   # to mark if the site is taken by a previous detected pair: >0: available   /   0: already taken
elv = []    # elevation of the site
exis = []   # to mark if the site is included in FLUXNET2015 with CC by 4.9
smdata=[]




pair_list=np.array([np.full((150,2),-999)]) # recording the pair by id
diag_distance=np.array([np.full((150,2),-999)]) # Calculating the distance between all site and a given site (as the reference) for making further site spatial visualization 
newid=np.array([np.full((150,2),-999)])   # Give the pair_list a id to make further site spatial visualization 
lonnew=np.array([np.full((150,2),-999)])  # latitude of the site of sites in pair_list
latnew=np.array([np.full((150,2),-999)])  # latitude of the site of sites in pair_list
with open(txtfile) as f:
    for line in f.readlines():
        s = line.split('\t')
        id.append((s[0]))
        name.append((s[1]))
        lat.append(float(s[2])+90)
        lon.append(float(s[3])+180)
        lct.append((s[4]))
        gns.append(int(s[5]))
        pool.append(int(s[0]))
        elv.append(float(s[6]))
        exis.append(int(s[7]))
miss_val=-9999999

dir = '/homes/eseo8/python/diurnal_cycle_sm/FLUXNET/CC-BY-4.0_202002/subset/FULLSET/'
site_list = sorted(glob.glob(dir+'*_FLUXNET2015_FULLSET_HR_*_CI_QC.csv'))
FLX_site = [None] * len(site_list)

for s,site in enumerate(site_list):
    # print (s,site.split("/")[9])
    FLX_site[s] = site.split("/")[9].split("_")[1]
del site_list
# FLX_site

dir = '/homes/eseo8/python/diurnal_cycle_sm/FLUXNET/Ameriflux/subset/FULLSET/'
site_list = sorted(glob.glob(dir+'*_CI_QC.csv'))
AMF_site = [None] * len(site_list)

for s,site in enumerate(site_list):
    # print (s,site.split("/")[9])
    AMF_site[s] = site.split("/")[9].split("_")[1]
del site_list
# AMF_site

dir = '/homes/eseo8/python/diurnal_cycle_sm/FLUXNET/Euroflux/subset/FULLSET/'
site_list = sorted(glob.glob(dir+'*_FLUXNET2015_FULLSET_HR_*_CI_QC.csv'))
EUF_site = [None] * len(site_list)

for s,site in enumerate(site_list):
    # print (s,site.split("/")[9])
    EUF_site[s] = site.split("/")[9].split("_")[1]
del site_list
# EUF_site

In [3]:
#Read file by globally seaching the file name with corresponding site name

import glob
import os
os.chdir('/shared/land/FluxTowers/FLUXNET2015/CC-BY-4.0_202002/FULLSET/HX')
"""
"""
miss_val=-9999999.9999999
Rdry=np.array([np.full((3,267),miss_val)])
Rwet=np.array([np.full((3,267),miss_val)])
SlopeSig=np.array([np.full((3,267),miss_val)])
Tbk=np.array([np.full((3,267),miss_val)])
SMbk=np.array([np.full((3,267),miss_val)])
Slopedry=np.array([np.full((3,267),miss_val)])
Slopewet=np.array([np.full((3,267),miss_val)])
SMbk_pt=np.array([np.full((3,267),miss_val)])
nsize=np.array([np.full((3,267),miss_val)])
iniyear=np.array([np.full((3,267),miss_val)])
endyear=np.array([np.full((3,267),miss_val)])
usemonth=np.array([np.full((3,267),miss_val)])
Tau=np.array([np.full((3,267),miss_val)])
SEdry=np.array([np.full((3,267),miss_val)])
SEwet=np.array([np.full((3,267),miss_val)])

guess_dry_slope, guess_wet_slope = -50.0, -25.0
#np.size(name)
for listn in range(0 ,np.size(name)):
    
    SM_siteA=[]
    H_siteA=[]
    LE_siteA=[]
    CO2_7_siteA=[]
    C02_9_siteA=[]
    TA_7_siteA=[]
    TA_9_siteA=[]
    TS_7_siteA=[]
    TS_9_siteA=[]
    if (listn>-1) :
        pass
        common_index=0
        siteA=name[listn]
        for file in glob.glob("FLX_" + siteA + "*"):
            print(file)
        csvfileA="/shared/land/FluxTowers/FLUXNET2015/CC-BY-4.0_202002/FULLSET/HX/" +file
      
        data_siteA = pd.read_csv(csvfileA) 
        #column_headers = list(data_siteB.columns.values)
        #print("The Column Header :", column_headers)

        #Tmax, ta_0700, ta_0900 

        if ('TA_F_MDS' in data_siteA.columns): 
            res = np.array(data_siteA['TIMESTAMP_START'].values.tolist())
            time=np.array(np.full((len(res)),"0000000000000000")) 

            for i in range(0 ,int(len(res))):
                a=str(res[i])
                time[i]= a[0:4] + "-" +  a[4:6] + "-" + a[6:8] 

            data_siteA['date'] = time
            data_siteA['date'] = pd.to_datetime(data_siteA['date'],  errors='coerce')           
            Tmin_max_siteA=data_siteA.groupby(pd.Grouper('date'))['TA_F_MDS'].agg(['max'])
            Tmin_max_siteA=Tmin_max_siteA['max'].replace(-9999.0, np.NaN)
            Tmin_max_siteA=Tmin_max_siteA.to_frame()
            
        if ('TS_F_MDS_1' in data_siteA.columns): 
            res = np.array(data_siteA['TIMESTAMP_START'].values.tolist())
            time=np.array(np.full((len(res)),"0000000000000000")) 

            for i in range(0 ,int(len(res))):
                a=str(res[i])
                time[i]= a[0:4] + "-" +  a[4:6] + "-" + a[6:8] 

            data_siteA['date'] = time
            data_siteA['date'] = pd.to_datetime(data_siteA['date'],  errors='coerce')           
            TS_max_siteA=data_siteA.groupby(pd.Grouper('date'))['TS_F_MDS_1'].agg(['max'])
            TS_max_siteA=TS_max_siteA['max'].replace(-9999.0, np.NaN)
            TS_max_siteA=TS_max_siteA.to_frame()
            
        if ('SWC_F_MDS_1' in data_siteA.columns): 
    
            res = np.array(data_siteA['TIMESTAMP_START'].values.tolist())
            time=np.array(np.full((len(res)),"0000000000000000")) 

            for i in range(0 ,int(len(res))):
                a=str(res[i])
                time[i]= a[8:10] + "" +  a[10:12]

            data_siteA['hour'] = time
            a=data_siteA[data_siteA["hour"] == '0900']['SWC_F_MDS_1']
            a.index=Tmin_max_siteA.index
            SM_siteA=a.to_frame()

            SM_siteA=SM_siteA['SWC_F_MDS_1'].replace(-9999.0, np.NaN)
            SM_siteA=SM_siteA.to_frame()
        
            Tmin_max_siteA=Tmin_max_siteA.dropna(subset=['max'])
            SM_siteA=SM_siteA.dropna(subset=['SWC_F_MDS_1'])
            common_index = set(Tmin_max_siteA.index).intersection(SM_siteA.index).intersection(TS_max_siteA.index)
            if len(common_index) > 0:
                try:
                    Tmin_max_siteA = Tmin_max_siteA.loc[common_index].copy()
                    Tmin_max_siteA = Tmin_max_siteA.sort_values(by='date')
                except:
                    pass
                try:

                    TS_max_siteA = TS_max_siteA.loc[common_index].copy()
                    TS_max_siteA = TS_max_siteA.sort_values(by='date')
                except:
                    pass
                try:

                    SM_siteA = SM_siteA.loc[common_index].copy()
                    SM_siteA = SM_siteA.sort_values(by='date')
                except:
                    pass
            try:
                Tmin_max_siteA_ano = Tmin_max_siteA.groupby(Tmin_max_siteA.index.month).transform(lambda g: g - g.mean())
                Tmin_max_siteA_cli = Tmin_max_siteA.groupby(Tmin_max_siteA.index.month).transform(lambda g:  g.mean())
                a=Tmin_max_siteA_cli.groupby(Tmin_max_siteA_cli.index.month)['max'].agg(['max']).to_numpy()
                x = np.where(a == a.max())[0]+1
                um=[x[0]-1,x[0],x[0]+1]
                if x[0]>11:
                    um=[11,12,1]
                if x[0]<2:    
                    um=[12,1,2]
                for i in range(0,3):
                    a = Tmin_max_siteA.index.month==um[i]    

                    guess_dry_slope, guess_wet_slope = -50.0, -25.0

                    lacc=SM_siteA[a].groupby(SM_siteA[a].index.year)['SWC_F_MDS_1'].apply(lambda x: x.autocorr(lag=1))
                    tau = -1/np.log(np.sqrt(np.sum(lacc*np.abs(lacc))/40))
                    dof=np.rint(float(len(SM_siteA[a]['SWC_F_MDS_1'])) / (tau + 1))

                    sm_sort=SM_siteA[a]['SWC_F_MDS_1'].to_numpy()
                    tx_sort=TS_max_siteA[a]['max'].to_numpy()

                    p , e = optimize.curve_fit(piecewise_linear, sm_sort, tx_sort, p0=[np.median(sm_sort),np.median(tx_sort),guess_dry_slope,guess_wet_slope], bounds=([np.min(sm_sort),np.min(tx_sort),-np.inf,-np.inf], [np.max(sm_sort),np.max(tx_sort),np.inf,np.inf]))
                    wet_pts = sum(i > p[0] for i in sm_sort) 
                    dry_pts = sum(i <= p[0] for i in sm_sort) 
                    perr = np.sqrt(np.diag(e))
                    vest = math.sqrt((tau+1)*(perr[2]*perr[2]/dry_pts+perr[3]*perr[3]/wet_pts)) # Adjust DOFs by tau, calculate z & p values for signif slope change
                    z = abs(p[2]-p[3])/vest
                    pval = st.norm.sf(abs(z))            
                    dry_corr, dry_corp = st.pearsonr(sm_sort[:dry_pts],tx_sort[:dry_pts])
                    wet_corr, wet_corp = st.pearsonr(sm_sort[-wet_pts:],tx_sort[-wet_pts:])



                    Rdry[0,i,listn]=dry_corr
                    Rwet[0,i,listn]=wet_corr
                    SlopeSig[0,i,listn]=pval
                    SMbk[0,i,listn]=p[0]
                    Tbk[0,i,listn]=p[1]
                    Slopedry[0,i,listn]=p[2]
                    Slopewet[0,i,listn]=p[3]
                    SMbk_pt[0,i,listn]=np.percentile(sm_sort,p[0]) 
                    nsize[0,i,listn]=np.size(sm_sort)
                    iniyear[0,i,listn]=SM_siteA[a].index.year[1]
                    endyear[0,i,listn]=SM_siteA[a].index.year[-1]                
                    usemonth[0,i,listn]=um[i] 
                    Tau[0,i,listn]=tau
                    SEdry[0,i,listn]=perr[2]*perr[2]/dry_pts 
                    SEwet[0,i,listn]=perr[3]*perr[3]/wet_pts              
            except:
                    pass

np.save('/homes/hhsu/02.InfoTheo/FluxNet/bl_sm_tsmax.npy', np.array([Rdry, Rwet, SlopeSig, Tbk, SMbk, Slopedry, Slopewet, SMbk_pt, usemonth, nsize, iniyear, endyear, Tau, SEdry, SEwet]))


FLX_AR-SLu_FLUXNET2015_FULLSET_HH_2009-2011_1-4.csv
FLX_AR-Vir_FLUXNET2015_FULLSET_HH_2009-2012_1-4.csv
FLX_AT-Neu_FLUXNET2015_FULLSET_HH_2002-2012_1-4.csv
FLX_AU-ASM_FLUXNET2015_FULLSET_HH_2010-2014_2-4.csv
FLX_AU-Ade_FLUXNET2015_FULLSET_HH_2007-2009_1-4.csv
FLX_AU-Cpr_FLUXNET2015_FULLSET_HH_2010-2014_2-4.csv
FLX_AU-Cum_FLUXNET2015_FULLSET_HH_2012-2014_2-4.csv
FLX_AU-DaP_FLUXNET2015_FULLSET_HH_2007-2013_2-4.csv
FLX_AU-DaS_FLUXNET2015_FULLSET_HH_2008-2014_2-4.csv
FLX_AU-Dry_FLUXNET2015_FULLSET_HH_2008-2014_2-4.csv
FLX_AU-Emr_FLUXNET2015_FULLSET_HH_2011-2013_1-4.csv
FLX_AU-Fog_FLUXNET2015_FULLSET_HH_2006-2008_1-4.csv
FLX_AU-GWW_FLUXNET2015_FULLSET_HH_2013-2014_1-4.csv
FLX_AU-Gin_FLUXNET2015_FULLSET_HH_2011-2014_1-4.csv
FLX_AU-How_FLUXNET2015_FULLSET_HH_2001-2014_1-4.csv
FLX_AU-Lox_FLUXNET2015_FULLSET_HH_2008-2009_1-4.csv
FLX_AU-RDF_FLUXNET2015_FULLSET_HH_2011-2013_1-4.csv
FLX_AU-Rig_FLUXNET2015_FULLSET_HH_2011-2014_2-4.csv
FLX_AU-Rob_FLUXNET2015_FULLSET_HH_2014-2014_1-4.csv
FLX_AU-Stp_F

In [7]:
#Read file by globally seaching the file name with corresponding site name

import glob
import os
"""
"""
miss_val=-9999999.9999999
Rdry=np.array([np.full((3,np.size(AMF_site)),miss_val)])
Rwet=np.array([np.full((3,np.size(AMF_site)),miss_val)])
SlopeSig=np.array([np.full((3,np.size(AMF_site)),miss_val)])
Tbk=np.array([np.full((3,np.size(AMF_site)),miss_val)])
SMbk=np.array([np.full((3,np.size(AMF_site)),miss_val)])
Slopedry=np.array([np.full((3,np.size(AMF_site)),miss_val)])
Slopewet=np.array([np.full((3,np.size(AMF_site)),miss_val)])
SMbk_pt=np.array([np.full((3,np.size(AMF_site)),miss_val)])
nsize=np.array([np.full((3,np.size(AMF_site)),miss_val)])
iniyear=np.array([np.full((3,np.size(AMF_site)),miss_val)])
endyear=np.array([np.full((3,np.size(AMF_site)),miss_val)])
usemonth=np.array([np.full((3,np.size(AMF_site)),miss_val)])
flagg=np.array([np.full((3,np.size(AMF_site)),miss_val)])

guess_dry_slope, guess_wet_slope = -50.0, -25.0
#np.size(name)
for listn in range(0 ,np.size(AMF_site)):
    
    SM_siteA=[]
    H_siteA=[]
    LE_siteA=[]
    CO2_7_siteA=[]
    C02_9_siteA=[]
    TA_7_siteA=[]
    TA_9_siteA=[]
    TS_7_siteA=[]
    TS_9_siteA=[]
    if (listn>-1) :
        pass
        common_index=0
        siteA=AMF_site[listn]
        aaa=len(glob.glob("/shared/land/FluxTowers/FLUXNET2015/CC-BY-4.0_202002/FULLSET/HX/FLX_" + siteA + "*"))
        bbb=len(glob.glob("/shared/land/FluxTowers/Ameriflux/HX/AMF_" + siteA + "*"))
        
        if aaa<1 and bbb>0:
            flagg[0,0,listn]=1
            
            for file in glob.glob("/shared/land/FluxTowers/Ameriflux/HX/AMF_" + siteA + "*"):
                print(file)
            csvfileA=file
            data_siteA = pd.read_csv(csvfileA,skiprows=2,na_values=-9999)  
            #column_headers = list(data_siteB.columns.values)
            #print("The Column Header :", column_headers)

            #Tmax, ta_0700, ta_0900 

            if ('TA' in data_siteA.columns): 
                res = np.array(data_siteA['TIMESTAMP_START'].values.tolist())
                time=np.array(np.full((len(res)),"0000000000000000")) 

                for i in range(0 ,int(len(res))):
                    a=str(res[i])
                    time[i]= a[0:4] + "-" +  a[4:6] + "-" + a[6:8] 

                data_siteA['date'] = time
                data_siteA['date'] = pd.to_datetime(data_siteA['date'],  errors='coerce')           
                Tmin_max_siteA=data_siteA.groupby(pd.Grouper('date'))['TA'].agg(['max'])
                Tmin_max_siteA=Tmin_max_siteA['max'].replace(-9999.0, np.NaN)
                Tmin_max_siteA=Tmin_max_siteA.to_frame()

            if ('TS_1' in data_siteA.columns): 
                res = np.array(data_siteA['TIMESTAMP_START'].values.tolist())
                time=np.array(np.full((len(res)),"0000000000000000")) 

                for i in range(0 ,int(len(res))):
                    a=str(res[i])
                    time[i]= a[0:4] + "-" +  a[4:6] + "-" + a[6:8] 

                data_siteA['date'] = time
                data_siteA['date'] = pd.to_datetime(data_siteA['date'],  errors='coerce')           
                TS_max_siteA=data_siteA.groupby(pd.Grouper('date'))['TS_1'].agg(['max'])
                TS_max_siteA=TS_max_siteA['max'].replace(-9999.0, np.NaN)
                TS_max_siteA=TS_max_siteA.to_frame()

            if ('SWC_1' in data_siteA.columns): 

                res = np.array(data_siteA['TIMESTAMP_START'].values.tolist())
                time=np.array(np.full((len(res)),"0000000000000000")) 

                for i in range(0 ,int(len(res))):
                    a=str(res[i])
                    time[i]= a[8:10] + "" +  a[10:12]

                data_siteA['hour'] = time
                a=data_siteA[data_siteA["hour"] == '0900']['SWC_1']
                a.index=Tmin_max_siteA.index
                SM_siteA=a.to_frame()

                SM_siteA=SM_siteA['SWC_1'].replace(-9999.0, np.NaN)
                SM_siteA=SM_siteA.to_frame()

                Tmin_max_siteA=Tmin_max_siteA.dropna(subset=['max'])
                SM_siteA=SM_siteA.dropna(subset=['SWC_1'])
                common_index = set(Tmin_max_siteA.index).intersection(SM_siteA.index).intersection(TS_max_siteA.index)
                if len(common_index) > 0:
                    try:
                        Tmin_max_siteA = Tmin_max_siteA.loc[common_index].copy()
                        Tmin_max_siteA = Tmin_max_siteA.sort_values(by='date')
                    except:
                        pass
                    try:

                        TS_max_siteA = TS_max_siteA.loc[common_index].copy()
                        TS_max_siteA = TS_max_siteA.sort_values(by='date')
                    except:
                        pass
                    try:

                        SM_siteA = SM_siteA.loc[common_index].copy()
                        SM_siteA = SM_siteA.sort_values(by='date')
                    except:
                        pass
                try:
                    Tmin_max_siteA_ano = Tmin_max_siteA.groupby(Tmin_max_siteA.index.month).transform(lambda g: g - g.mean())
                    Tmin_max_siteA_cli = Tmin_max_siteA.groupby(Tmin_max_siteA.index.month).transform(lambda g:  g.mean())
                    a=Tmin_max_siteA_cli.groupby(Tmin_max_siteA_cli.index.month)['max'].agg(['max']).to_numpy()
                    x = np.where(a == a.max())[0]+1
                    um=[x[0]-1,x[0],x[0]+1]
                    if x[0]>11:
                        um=[11,12,1]
                    if x[0]<2:    
                        um=[12,1,2]
                    for i in range(0,3):
                        a = Tmin_max_siteA.index.month==um[i]    

                        guess_dry_slope, guess_wet_slope = -50.0, -25.0

                        lacc=SM_siteA[a].groupby(SM_siteA[a].index.year)['SWC_1'].apply(lambda x: x.autocorr(lag=1))
                        tau = -1/np.log(np.sqrt(np.sum(lacc*np.abs(lacc))/40))
                        dof=np.rint(float(len(SM_siteA[a]['SWC_1'])) / (tau + 1))

                        sm_sort=SM_siteA[a]['SWC_1'].to_numpy()
                        tx_sort=TS_max_siteA[a]['max'].to_numpy()

                        p , e = optimize.curve_fit(piecewise_linear, sm_sort, tx_sort, p0=[np.median(sm_sort),np.median(tx_sort),guess_dry_slope,guess_wet_slope], bounds=([np.min(sm_sort),np.min(tx_sort),-np.inf,-np.inf], [np.max(sm_sort),np.max(tx_sort),np.inf,np.inf]))
                        wet_pts = sum(i > p[0] for i in sm_sort) 
                        dry_pts = sum(i <= p[0] for i in sm_sort) 
                        perr = np.sqrt(np.diag(e))
                        vest = math.sqrt((tau+1)*(perr[2]*perr[2]/dry_pts+perr[3]*perr[3]/wet_pts)) # Adjust DOFs by tau, calculate z & p values for signif slope change
                        z = abs(p[2]-p[3])/vest
                        pval = st.norm.sf(abs(z))            
                        dry_corr, dry_corp = st.pearsonr(sm_sort[:dry_pts],tx_sort[:dry_pts])
                        wet_corr, wet_corp = st.pearsonr(sm_sort[-wet_pts:],tx_sort[-wet_pts:])



                        Rdry[0,i,listn]=dry_corr
                        Rwet[0,i,listn]=wet_corr
                        SlopeSig[0,i,listn]=pval
                        SMbk[0,i,listn]=p[0]
                        Tbk[0,i,listn]=p[1]
                        Slopedry[0,i,listn]=p[2]
                        Slopewet[0,i,listn]=p[3]
                        SMbk_pt[0,i,listn]=np.percentile(sm_sort,p[0]) 
                        nsize[0,i,listn]=np.size(sm_sort)
                        iniyear[0,i,listn]=SM_siteA[a].index.year[1]
                        endyear[0,i,listn]=SM_siteA[a].index.year[-1]
                        usemonth[0,i,listn]=um[i] 

                except:
                        pass

np.save('/homes/hhsu/02.InfoTheo/FluxNet/bl_sm_tsmax_Amer.npy', np.array([Rdry, Rwet, SlopeSig, Tbk, SMbk, Slopedry, Slopewet,SMbk_pt, usemonth, nsize, iniyear, endyear, flagg]))


/shared/land/FluxTowers/Ameriflux/HX/AMF_AR-TF1_BASE_HH_1-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_BR-Npw_BASE_HH_1-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-ARB_BASE_HH_1-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-ARF_BASE_HH_1-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-Ca1_BASE_HH_1-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-Ca2_BASE_HH_1-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-Ca3_BASE_HH_4-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-Cbo_BASE_HH_3-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-DBB_BASE_HH_1-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-ER1_BASE_HH_3-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-Let_BASE_HH_1-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-MA2_BASE_HH_1-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-MA3_BASE_HH_1-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-Ojp_BASE_HH_2-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF_CA-Qc2_BASE_HH_1-5.csv
/shared/land/FluxTowers/Ameriflux/HX/AMF

In [28]:
AMF_site

['AR-TF1',
 'BR-Npw',
 'BR-Sa1',
 'BR-Sa3',
 'CA-ARB',
 'CA-ARF',
 'CA-Ca1',
 'CA-Ca2',
 'CA-Ca3',
 'CA-Cbo',
 'CA-DBB',
 'CA-ER1',
 'CA-Gro',
 'CA-Let',
 'CA-MA2',
 'CA-MA3',
 'CA-Man',
 'CA-NS1',
 'CA-NS2',
 'CA-NS3',
 'CA-NS4',
 'CA-NS5',
 'CA-NS6',
 'CA-NS7',
 'CA-Oas',
 'CA-Obs',
 'CA-Ojp',
 'CA-Qc2',
 'CA-Qcu',
 'CA-Qfo',
 'CA-SCC',
 'CA-SF1',
 'CA-SF2',
 'CA-SF3',
 'CA-SJ1',
 'CA-SJ2',
 'CA-SJ3',
 'CA-TP1',
 'CA-TP2',
 'CA-TP3',
 'CA-TP4',
 'CA-TPD',
 'CA-WP1',
 'CR-Lse',
 'MX-Aog',
 'MX-EMg',
 'MX-Lpa',
 'MX-Tes',
 'US-ADR',
 'US-ALQ',
 'US-AR1',
 'US-AR2',
 'US-ARM',
 'US-Atq',
 'US-Aud',
 'US-BRG',
 'US-BZB',
 'US-BZF',
 'US-Bar',
 'US-Bi1',
 'US-Bi2',
 'US-Bkg',
 'US-Blk',
 'US-Blo',
 'US-Bn1',
 'US-Bn2',
 'US-Bn3',
 'US-Bo1',
 'US-Br1',
 'US-Br3',
 'US-CMW',
 'US-CPk',
 'US-CZ2',
 'US-CZ3',
 'US-CaV',
 'US-Ced',
 'US-ChR',
 'US-Cop',
 'US-Ctn',
 'US-Cwt',
 'US-DPW',
 'US-Dix',
 'US-Dk1',
 'US-Dk2',
 'US-Dk3',
 'US-EML',
 'US-FPe',
 'US-FR2',
 'US-FR3',
 'US-Fcr',
 'US-Fmf',

In [17]:
#Read file by globally seaching the file name with corresponding site name

import glob
import os
"""
"""
miss_val=-9999999.9999999
Rdry=np.array([np.full((3,np.size(AMF_site)),miss_val)])
Rwet=np.array([np.full((3,np.size(AMF_site)),miss_val)])
SlopeSig=np.array([np.full((3,np.size(AMF_site)),miss_val)])
Tbk=np.array([np.full((3,np.size(AMF_site)),miss_val)])
SMbk=np.array([np.full((3,np.size(AMF_site)),miss_val)])
Slopedry=np.array([np.full((3,np.size(AMF_site)),miss_val)])
Slopewet=np.array([np.full((3,np.size(AMF_site)),miss_val)])
SMbk_pt=np.array([np.full((3,np.size(AMF_site)),miss_val)])
nsize=np.array([np.full((3,np.size(AMF_site)),miss_val)])
iniyear=np.array([np.full((3,np.size(AMF_site)),miss_val)])
endyear=np.array([np.full((3,np.size(AMF_site)),miss_val)])
usemonth=np.array([np.full((3,np.size(AMF_site)),miss_val)])
flagg=np.array([np.full((3,np.size(AMF_site)),miss_val)])

guess_dry_slope, guess_wet_slope = -50.0, -25.0
#np.size(name)
for listn in range(0 ,np.size(EUF_site)):
    
    SM_siteA=[]
    H_siteA=[]
    LE_siteA=[]
    CO2_7_siteA=[]
    C02_9_siteA=[]
    TA_7_siteA=[]
    TA_9_siteA=[]
    TS_7_siteA=[]
    TS_9_siteA=[]
    if (listn>-1) :
        pass
        common_index=0
        siteA=EUF_site[listn]
        aaa=len(glob.glob("/shared/land/FluxTowers/FLUXNET2015/CC-BY-4.0_202002/FULLSET/HX/FLX_" + siteA + "*"))
        bbb=len(glob.glob("/shared/land/FluxTowers/FLUXNET2015/Europe_2018/FULLSET/FLX_" + siteA + "_FLUXNET2015_FULLSET_HH*"))
        
        if aaa<1 and bbb>0:
            flagg[0,0,listn]=1
            for file in glob.glob("/shared/land/FluxTowers/FLUXNET2015/Europe_2018/FULLSET/FLX_" + siteA + "_FLUXNET2015_FULLSET_HH*"):
                print(file)
            csvfileA=file
            data_siteA = pd.read_csv(csvfileA)            #column_headers = list(data_siteB.columns.values)
            #print("The Column Header :", column_headers)

            #Tmax, ta_0700, ta_0900 

            if ('TA_F_MDS' in data_siteA.columns): 
                res = np.array(data_siteA['TIMESTAMP_START'].values.tolist())
                time=np.array(np.full((len(res)),"0000000000000000")) 

                for i in range(0 ,int(len(res))):
                    a=str(res[i])
                    time[i]= a[0:4] + "-" +  a[4:6] + "-" + a[6:8] 

                data_siteA['date'] = time
                data_siteA['date'] = pd.to_datetime(data_siteA['date'],  errors='coerce')           
                Tmin_max_siteA=data_siteA.groupby(pd.Grouper('date'))['TA_F_MDS'].agg(['max'])
                Tmin_max_siteA=Tmin_max_siteA['max'].replace(-9999.0, np.NaN)
                Tmin_max_siteA=Tmin_max_siteA.to_frame()

            if ('TS_F_MDS_1' in data_siteA.columns): 
                res = np.array(data_siteA['TIMESTAMP_START'].values.tolist())
                time=np.array(np.full((len(res)),"0000000000000000")) 

                for i in range(0 ,int(len(res))):
                    a=str(res[i])
                    time[i]= a[0:4] + "-" +  a[4:6] + "-" + a[6:8] 

                data_siteA['date'] = time
                data_siteA['date'] = pd.to_datetime(data_siteA['date'],  errors='coerce')           
                TS_max_siteA=data_siteA.groupby(pd.Grouper('date'))['TS_F_MDS_1'].agg(['max'])
                TS_max_siteA=TS_max_siteA['max'].replace(-9999.0, np.NaN)
                TS_max_siteA=TS_max_siteA.to_frame()

            if ('SWC_F_MDS_1' in data_siteA.columns): 

                res = np.array(data_siteA['TIMESTAMP_START'].values.tolist())
                time=np.array(np.full((len(res)),"0000000000000000")) 

                for i in range(0 ,int(len(res))):
                    a=str(res[i])
                    time[i]= a[8:10] + "" +  a[10:12]

                data_siteA['hour'] = time
                a=data_siteA[data_siteA["hour"] == '0900']['SWC_F_MDS_1']
                a.index=Tmin_max_siteA.index
                SM_siteA=a.to_frame()

                SM_siteA=SM_siteA['SWC_F_MDS_1'].replace(-9999.0, np.NaN)
                SM_siteA=SM_siteA.to_frame()

                Tmin_max_siteA=Tmin_max_siteA.dropna(subset=['max'])
                SM_siteA=SM_siteA.dropna(subset=['SWC_F_MDS_1'])
                common_index = set(Tmin_max_siteA.index).intersection(SM_siteA.index).intersection(TS_max_siteA.index)
                if len(common_index) > 0:
                    try:
                        Tmin_max_siteA = Tmin_max_siteA.loc[common_index].copy()
                        Tmin_max_siteA = Tmin_max_siteA.sort_values(by='date')
                    except:
                        pass
                    try:

                        TS_max_siteA = TS_max_siteA.loc[common_index].copy()
                        TS_max_siteA = TS_max_siteA.sort_values(by='date')
                    except:
                        pass
                    try:

                        SM_siteA = SM_siteA.loc[common_index].copy()
                        SM_siteA = SM_siteA.sort_values(by='date')
                    except:
                        pass
                try:
                    Tmin_max_siteA_ano = Tmin_max_siteA.groupby(Tmin_max_siteA.index.month).transform(lambda g: g - g.mean())
                    Tmin_max_siteA_cli = Tmin_max_siteA.groupby(Tmin_max_siteA.index.month).transform(lambda g:  g.mean())
                    a=Tmin_max_siteA_cli.groupby(Tmin_max_siteA_cli.index.month)['max'].agg(['max']).to_numpy()
                    x = np.where(a == a.max())[0]+1
                    um=[x[0]-1,x[0],x[0]+1]
                    if x[0]>11:
                        um=[11,12,1]
                    if x[0]<2:    
                        um=[12,1,2]
                    for i in range(0,3):
                        a = Tmin_max_siteA.index.month==um[i]    

                        guess_dry_slope, guess_wet_slope = -50.0, -25.0

                        lacc=SM_siteA[a].groupby(SM_siteA[a].index.year)['SWC_F_MDS_1'].apply(lambda x: x.autocorr(lag=1))
                        tau = -1/np.log(np.sqrt(np.sum(lacc*np.abs(lacc))/40))
                        dof=np.rint(float(len(SM_siteA[a]['SWC_F_MDS_1'])) / (tau + 1))

                        sm_sort=SM_siteA[a]['SWC_F_MDS_1'].to_numpy()
                        tx_sort=Tmin_max_siteA[a]['max'].to_numpy()

                        p , e = optimize.curve_fit(piecewise_linear, sm_sort, tx_sort, p0=[np.median(sm_sort),np.median(tx_sort),guess_dry_slope,guess_wet_slope], bounds=([np.min(sm_sort),np.min(tx_sort),-np.inf,-np.inf], [np.max(sm_sort),np.max(tx_sort),np.inf,np.inf]))
                        wet_pts = sum(i > p[0] for i in sm_sort) 
                        dry_pts = sum(i <= p[0] for i in sm_sort) 
                        perr = np.sqrt(np.diag(e))
                        vest = math.sqrt((tau+1)*(perr[2]*perr[2]/dry_pts+perr[3]*perr[3]/wet_pts)) # Adjust DOFs by tau, calculate z & p values for signif slope change
                        z = abs(p[2]-p[3])/vest
                        pval = st.norm.sf(abs(z))            
                        dry_corr, dry_corp = st.pearsonr(sm_sort[:dry_pts],tx_sort[:dry_pts])
                        wet_corr, wet_corp = st.pearsonr(sm_sort[-wet_pts:],tx_sort[-wet_pts:])



                        Rdry[0,i,listn]=dry_corr
                        Rwet[0,i,listn]=wet_corr
                        SlopeSig[0,i,listn]=pval
                        SMbk[0,i,listn]=p[0]
                        Tbk[0,i,listn]=p[1]
                        Slopedry[0,i,listn]=p[2]
                        Slopewet[0,i,listn]=p[3]
                        SMbk_pt[0,i,listn]=np.percentile(sm_sort,p[0]) 
                        nsize[0,i,listn]=np.size(sm_sort)
                        iniyear[0,i,listn]=SM_siteA[a].index.year[1]
                        endyear[0,i,listn]=SM_siteA[a].index.year[-1]                
                        usemonth[0,i,listn]=um[i] 
                        
                except:
                        pass
np.save('/homes/hhsu/02.InfoTheo/FluxNet/bl_sm_tsmax_Euf.npy', np.array([Rdry, Rwet, SlopeSig, Tbk, SMbk, Slopedry, Slopewet,SMbk_pt, usemonth, nsize, iniyear, endyear, flagg]))


/shared/land/FluxTowers/FLUXNET2015/Europe_2018/FULLSET/FLX_CH-Aws_FLUXNET2015_FULLSET_HH_2010-2018_beta-3.csv
/shared/land/FluxTowers/FLUXNET2015/Europe_2018/FULLSET/FLX_CZ-Lnz_FLUXNET2015_FULLSET_HH_2015-2018_beta-3.csv
/shared/land/FluxTowers/FLUXNET2015/Europe_2018/FULLSET/FLX_CZ-RAJ_FLUXNET2015_FULLSET_HH_2012-2018_beta-3.csv
/shared/land/FluxTowers/FLUXNET2015/Europe_2018/FULLSET/FLX_CZ-Stn_FLUXNET2015_FULLSET_HH_2010-2018_beta-3.csv
/shared/land/FluxTowers/FLUXNET2015/Europe_2018/FULLSET/FLX_DE-HoH_FLUXNET2015_FULLSET_HH_2015-2018_beta-3.csv
/shared/land/FluxTowers/FLUXNET2015/Europe_2018/FULLSET/FLX_DE-Hte_FLUXNET2015_FULLSET_HH_2009-2018_beta-3.csv
/shared/land/FluxTowers/FLUXNET2015/Europe_2018/FULLSET/FLX_DE-Hzd_FLUXNET2015_FULLSET_HH_2010-2018_beta-3.csv
/shared/land/FluxTowers/FLUXNET2015/Europe_2018/FULLSET/FLX_DE-RuW_FLUXNET2015_FULLSET_HH_2010-2018_beta-3.csv
/shared/land/FluxTowers/FLUXNET2015/Europe_2018/FULLSET/FLX_ES-Abr_FLUXNET2015_FULLSET_HH_2015-2018_beta-3.csv
/

In [54]:
flagg

array([[[-1.e+07, -1.e+07, -1.e+07,  1.e+00, -1.e+07, -1.e+07, -1.e+07,
         -1.e+07, -1.e+07, -1.e+07,  1.e+00,  1.e+00,  1.e+00, -1.e+07,
         -1.e+07, -1.e+07, -1.e+07, -1.e+07,  1.e+00,  1.e+00,  1.e+00,
         -1.e+07, -1.e+07, -1.e+07, -1.e+07,  1.e+00, -1.e+07, -1.e+07,
          1.e+00,  1.e+00,  1.e+00, -1.e+07, -1.e+07,  1.e+00,  1.e+00,
          1.e+00,  1.e+00,  1.e+00, -1.e+07, -1.e+07,  1.e+00, -1.e+07,
         -1.e+07, -1.e+07,  1.e+00, -1.e+07,  1.e+00,  1.e+00,  1.e+00,
          1.e+00,  1.e+00,  1.e+00, -1.e+07, -1.e+07, -1.e+07, -1.e+07,
         -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07,
         -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07,
         -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07,
         -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07,
         -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07,
         -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07, -1.e+07, -

In [15]:
data_siteA

,TIMESTAMP_START,TIMESTAMP_END,TA_F_MDS,TA_F_MDS_QC,TA_ERA,TA_F,TA_F_QC,SW_IN_POT,SW_IN_F_MDS,SW_IN_F_MDS_QC,...,GPP_DT_CUT_SE,GPP_DT_CUT_05,GPP_DT_CUT_16,GPP_DT_CUT_25,GPP_DT_CUT_50,GPP_DT_CUT_75,GPP_DT_CUT_84,GPP_DT_CUT_95,RECO_SR,RECO_SR_N
0,201401010000,201401010030,-0.412,0,0.401,-0.412,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999,-9999
1,201401010030,201401010100,-0.455,0,0.376,-0.455,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999,-9999
2,201401010100,201401010130,-0.574,0,0.352,-0.574,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999,-9999
3,201401010130,201401010200,-0.597,0,0.160,-0.597,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999,-9999
4,201401010200,201401010230,-0.514,0,-0.032,-0.514,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999,-9999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87643,201812312130,201812312200,-0.484,0,3.110,-0.484,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999,-9999
87644,201812312200,201812312230,-0.269,0,3.552,-0.269,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999,-9999
87645,201812312230,201812312300,-0.266,0,3.398,-0.266,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999,-9999
87646,201812312300,201812312330,-0.079,0,3.244,-0.079,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9999,-9999


In [16]:
data_siteA.columns

Index(['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F_MDS', 'TA_F_MDS_QC', 'TA_ERA',
       'TA_F', 'TA_F_QC', 'SW_IN_POT', 'SW_IN_F_MDS', 'SW_IN_F_MDS_QC',
       ...
       'GPP_DT_CUT_SE', 'GPP_DT_CUT_05', 'GPP_DT_CUT_16', 'GPP_DT_CUT_25',
       'GPP_DT_CUT_50', 'GPP_DT_CUT_75', 'GPP_DT_CUT_84', 'GPP_DT_CUT_95',
       'RECO_SR', 'RECO_SR_N'],
      dtype='object', length=231)

In [22]:
'Latitude' in data_siteA.columns

False

In [24]:
AMF_site

['AR-TF1',
 'BR-Npw',
 'BR-Sa1',
 'BR-Sa3',
 'CA-ARB',
 'CA-ARF',
 'CA-Ca1',
 'CA-Ca2',
 'CA-Ca3',
 'CA-Cbo',
 'CA-DBB',
 'CA-ER1',
 'CA-Gro',
 'CA-Let',
 'CA-MA2',
 'CA-MA3',
 'CA-Man',
 'CA-NS1',
 'CA-NS2',
 'CA-NS3',
 'CA-NS4',
 'CA-NS5',
 'CA-NS6',
 'CA-NS7',
 'CA-Oas',
 'CA-Obs',
 'CA-Ojp',
 'CA-Qc2',
 'CA-Qcu',
 'CA-Qfo',
 'CA-SCC',
 'CA-SF1',
 'CA-SF2',
 'CA-SF3',
 'CA-SJ1',
 'CA-SJ2',
 'CA-SJ3',
 'CA-TP1',
 'CA-TP2',
 'CA-TP3',
 'CA-TP4',
 'CA-TPD',
 'CA-WP1',
 'CR-Lse',
 'MX-Aog',
 'MX-EMg',
 'MX-Lpa',
 'MX-Tes',
 'US-ADR',
 'US-ALQ',
 'US-AR1',
 'US-AR2',
 'US-ARM',
 'US-Atq',
 'US-Aud',
 'US-BRG',
 'US-BZB',
 'US-BZF',
 'US-Bar',
 'US-Bi1',
 'US-Bi2',
 'US-Bkg',
 'US-Blk',
 'US-Blo',
 'US-Bn1',
 'US-Bn2',
 'US-Bn3',
 'US-Bo1',
 'US-Br1',
 'US-Br3',
 'US-CMW',
 'US-CPk',
 'US-CZ2',
 'US-CZ3',
 'US-CaV',
 'US-Ced',
 'US-ChR',
 'US-Cop',
 'US-Ctn',
 'US-Cwt',
 'US-DPW',
 'US-Dix',
 'US-Dk1',
 'US-Dk2',
 'US-Dk3',
 'US-EML',
 'US-FPe',
 'US-FR2',
 'US-FR3',
 'US-Fcr',
 'US-Fmf',

In [30]:

txtfile = '/homes/hhsu/02.InfoTheo/FluxNet/AmeriFlux_site_Info.txt'
id = []     # The number of the site
name = []   # The name of the site
lat = []    # latitude of the site
lon = []    # lontitude of the site


with open(txtfile) as f:
    for line in f.readlines():
        s = line.split('\t')
        id.append((s[0]))
        name.append((s[1]))
        lat.append(float(s[2])+90)
        lon.append(float(s[3])+180)
       


In [31]:
name

['AR-CCa',
 'AR-CCg',
 'AR-TF1',
 'AR-TF2',
 'BR-CMT',
 'BR-CST',
 'BR-Cui',
 'BR-Ma2',
 'BR-Npw',
 'BR-PRS',
 'BR-Sa1',
 'BR-Sa3',
 'CA-ARB',
 'CA-ARF',
 'CA-BOU',
 'CA-Ca1',
 'CA-Ca2',
 'CA-Ca3',
 'CA-Cbo',
 'CA-CF1',
 'CA-CF2',
 'CA-Cha',
 'CA-DB2',
 'CA-DBB',
 'CA-DL1',
 'CA-DL2',
 'CA-DSM',
 'CA-ER1',
 'CA-ER2',
 'CA-Gro',
 'CA-HPC',
 'CA-KLP',
 'CA-Let',
 'CA-LP1',
 'CA-LuM',
 'CA-MA1',
 'CA-MA2',
 'CA-MA3',
 'CA-Man',
 'CA-Mer',
 'CA-MR3',
 'CA-MR5',
 'CA-Na1',
 'CA-NS1',
 'CA-NS2',
 'CA-NS3',
 'CA-NS4',
 'CA-NS5',
 'CA-NS6',
 'CA-NS7',
 'CA-NS8',
 'CA-Oas',
 'CA-Obs',
 'CA-Ojp',
 'CA-Qc2',
 'CA-Qcu',
 'CA-Qfo',
 'CA-RBM',
 'CA-RPn',
 'CA-RPp',
 'CA-SCB',
 'CA-SCC',
 'CA-SF1',
 'CA-SF2',
 'CA-SF3',
 'CA-SJ1',
 'CA-SJ2',
 'CA-SJ3',
 'CA-SMC',
 'CA-TP1',
 'CA-TP2',
 'CA-TP3',
 'CA-TP4',
 'CA-TPD',
 'CA-TVC',
 'CA-WP1',
 'CA-WP2',
 'CA-WP3',
 'CL-ACF',
 'CL-SDF',
 'CL-SDP',
 'CO-GV1',
 'CO-GV2',
 'CR-Dos',
 'CR-Fsc',
 'CR-Lse',
 'CR-Nmr',
 'CR-SoC',
 'MX-Aog',
 'MX-Asf',
 'MX-BRN',

In [52]:
name.index(AMF_site[0])
lon[name.index(AMF_site[0])]

113.2665

In [43]:
AMF_site[0]

'AR-TF1'

In [51]:
type(name.index(AMF_site[0]))

int

In [46]:
type(AMF_site[1])

str